In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import sys
import re
import unicodedata
import pandas as pd

In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)
soup = BeautifulSoup(response.text, 'html.parser')
soup.title
first_launch_table = soup.find_all('tbody')[2]
print(first_launch_table)

In [50]:
column_names = []
for col in first_launch_table.find_all('th'):
    column_names.append(col.text.strip('\n'))
removal = ['1', '2', '3', '4', '5', '6', '7']
for remove in removal:
    column_names.remove(remove)
print(column_names)
print(len(column_names))


['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding']
10


In [10]:
launch_dict = dict.fromkeys(column_names)
launch_dict#for key in launch_dict.keys():

{'Flight No.': None,
 'Date andtime (UTC)': None,
 'Version,Booster [b]': None,
 'Launch site': None,
 'Payload[c]': None,
 'Payload mass': None,
 'Orbit': None,
 'Customer': None,
 'Launchoutcome': None,
 'Boosterlanding': None}

In [19]:
scrapeResult = {}
counter = 0
for row in first_launch_table.find_all('tr'):
    data = row.find_all('td')
    if len(data) == 9:
        rowResult = []
        counter += 1
        for value in data:
            rowResult.append(value.text.strip('\n'))
        scrapeResult[counter] = rowResult
        



In [20]:
print(scrapeResult)

{1: ['4 June 2010,18:45', 'F9 v1.0[7]B0003.1[8]', 'CCAFS,SLC-40', 'Dragon Spacecraft Qualification Unit', '', 'LEO', 'SpaceX', 'Success', 'Failure[9][10](parachute)'], 2: ['8 December 2010,15:43[13]', 'F9 v1.0[7]B0004.1[8]', 'CCAFS,SLC-40', 'Dragon demo flight C1(Dragon C101)', '', 'LEO (ISS)', 'NASA (COTS)\nNRO', 'Success[9]', 'Failure[9][14](parachute)'], 3: ['22 May 2012,07:44[17]', 'F9 v1.0[7]B0005.1[8]', 'CCAFS,SLC-40', 'Dragon demo flight C2+[18](Dragon C102)', '525\xa0kg (1,157\xa0lb)[19]', 'LEO (ISS)', 'NASA (COTS)', 'Success[20]', 'No attempt'], 4: ['8 October 2012,00:35[21]', 'F9 v1.0[7]B0006.1[8]', 'CCAFS,SLC-40', 'SpaceX CRS-1[22](Dragon C103)', '4,700\xa0kg (10,400\xa0lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt'], 5: ['1 March 2013,15:10', 'F9 v1.0[7]B0007.1[8]', 'CCAFS,SLC-40', 'SpaceX CRS-2[22](Dragon C104)', '4,877\xa0kg (10,752\xa0lb)', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt'], 6: ['29 September 2013,16:00[30]', 'F9 v1.1[7]B1003[8]', 'VAFB,SLC-4E

In [21]:
for key, theList in scrapeResult.items():
    theList.insert(0, key)


In [22]:
scrapeResult

{1: [1,
  '4 June 2010,18:45',
  'F9 v1.0[7]B0003.1[8]',
  'CCAFS,SLC-40',
  'Dragon Spacecraft Qualification Unit',
  '',
  'LEO',
  'SpaceX',
  'Success',
  'Failure[9][10](parachute)'],
 2: [2,
  '8 December 2010,15:43[13]',
  'F9 v1.0[7]B0004.1[8]',
  'CCAFS,SLC-40',
  'Dragon demo flight C1(Dragon C101)',
  '',
  'LEO (ISS)',
  'NASA (COTS)\nNRO',
  'Success[9]',
  'Failure[9][14](parachute)'],
 3: [3,
  '22 May 2012,07:44[17]',
  'F9 v1.0[7]B0005.1[8]',
  'CCAFS,SLC-40',
  'Dragon demo flight C2+[18](Dragon C102)',
  '525\xa0kg (1,157\xa0lb)[19]',
  'LEO (ISS)',
  'NASA (COTS)',
  'Success[20]',
  'No attempt'],
 4: [4,
  '8 October 2012,00:35[21]',
  'F9 v1.0[7]B0006.1[8]',
  'CCAFS,SLC-40',
  'SpaceX CRS-1[22](Dragon C103)',
  '4,700\xa0kg (10,400\xa0lb)',
  'LEO (ISS)',
  'NASA (CRS)',
  'Success',
  'No attempt'],
 5: [5,
  '1 March 2013,15:10',
  'F9 v1.0[7]B0007.1[8]',
  'CCAFS,SLC-40',
  'SpaceX CRS-2[22](Dragon C104)',
  '4,877\xa0kg (10,752\xa0lb)',
  'LEO (ISS)',
  'NAS

In [42]:
outerList = []
for innerCounter in range(0, 10):
    innerList=[]
    for outerCounter in range(1, 8):
        innerList.append(scrapeResult[outerCounter][innerCounter])
    outerList.append(innerList)
    


In [45]:
counter = 0
for key in launch_dict.keys():
    launch_dict[key] = outerList[counter]
    counter += 1

In [47]:
df = pd.DataFrame(launch_dict)
df

,Flight No.,Date andtime (UTC),"Version,Booster [b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding
0,1,"4 June 2010,18:45",F9 v1.0[7]B0003.1[8],"CCAFS,SLC-40",Dragon Spacecraft Qualification Unit,,LEO,SpaceX,Success,Failure[9][10](parachute)
1,2,"8 December 2010,15:43[13]",F9 v1.0[7]B0004.1[8],"CCAFS,SLC-40",Dragon demo flight C1(Dragon C101),,LEO (ISS),NASA (COTS)\nNRO,Success[9],Failure[9][14](parachute)
2,3,"22 May 2012,07:44[17]",F9 v1.0[7]B0005.1[8],"CCAFS,SLC-40",Dragon demo flight C2+[18](Dragon C102),"525 kg (1,157 lb)[19]",LEO (ISS),NASA (COTS),Success[20],No attempt
3,4,"8 October 2012,00:35[21]",F9 v1.0[7]B0006.1[8],"CCAFS,SLC-40",SpaceX CRS-1[22](Dragon C103),"4,700 kg (10,400 lb)",LEO (ISS),NASA (CRS),Success,No attempt
4,5,"1 March 2013,15:10",F9 v1.0[7]B0007.1[8],"CCAFS,SLC-40",SpaceX CRS-2[22](Dragon C104),"4,877 kg (10,752 lb)",LEO (ISS),NASA (CRS),Success,No attempt
5,6,"29 September 2013,16:00[30]",F9 v1.1[7]B1003[8],"VAFB,SLC-4E",CASSIOPE[22][31],"500 kg (1,100 lb)",Polar orbit LEO,MDA,Success[30],Uncontrolled(ocean)[d]
6,7,"3 December 2013,22:41[34]",F9 v1.1B1004,"CCAFS,SLC-40",SES-8[22][35][36],"3,170 kg (6,990 lb)",GTO,SES,Success[37],No attempt[38]
